In [ ]:
import os
import datetime as dt
import time
import pytz
import calendar
import pandas as pd

from uce_resources import get_site_id, get_mms_data, get_applied_forecast

In [ ]:
from settings.sites import ceg as sites_list

target_year = 2023
target_month = 8

target_folder = 'data/forecasts/month_forecast/'
if not os.path.exists(target_folder):
    os.makedirs(target_folder)

sites_list = ['Pohrebyshche', 'Bilashky',
                'Bar', 'Balky', 'Verkhivka', 'Kopaihorod', 'Stanislavchyk', 'Sharhorod_1', 'Hnatkiv', 'Porohy',
                'Hlybochok_1', 'Hlybochok_2.1', 'Hlybochok_2.2', 'Cherniatka','Chechelnyk_1', 'Chechelnyk_2',
                'Komyshany_1', 'Komyshany_2', 'Bilozerka', 'Myroliubivka', 'Kyselivka',
                'Oleshky_1', 'Oleshky_2', 'Poniativka', 'Kostohryzove', 
                'Veliton', 'Velihen', 
                'Mykolaivka', 'Rubanivka', 'Mala_Lepetykha', 'Kachkarivka', 
                'Liubymivka', 'Vasylivka',
                'Dibrovka', 'Yelanets_1', 'Yelanets_2', 
                'Afanasiivka', 'Novokondakove', 'Bazaltova', 'Inhulets_1', 'Inhulets_2', 'Bereznehuvate', 
                'Teplychna', 
                'Balivka', 'Solone', 'Stepnohirsk', 
                'Kulevcha']
sites_data = dict.fromkeys(sites_list)
print(sites_data)

In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.pool import NullPool
from settings.db import DO_URL

engine = create_engine(DO_URL, poolclass=NullPool)
metadata = MetaData()
metadata.reflect(bind=engine)

Applied Forecast

In [ ]:
with engine.connect() as connection:

    forecast_list = list()
    consumption_forecast_list = list()

    for site in sites_data.keys():
        start = time.time()
        print('-'*50)
        print(site)
        site_data = dict()
        site_data['site'] = site
        site_data['site_id'], site_data['legal_entity'] = get_site_id(site, connection, 
                                                                      metadata.tables['sites'],
                                                                      include_legal_entity_id=True)
        mms=pd.DataFrame()
   
        first_date = dt.date(target_year, target_month, 1),
        last_date = dt.date(
            target_year, 
            target_month, 
            calendar.monthrange(target_year, target_month)[1]
        ) + dt.timedelta(days=7)

        applied_forecast = get_applied_forecast(
            site_data['site_id'], 
            first_date,
            last_date,
            connection=connection, 
            db_table=metadata.tables['forecasting_data']
        )

        applied_forecast.index = applied_forecast.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev'))
        # print(applied_forecast)
        applied_forecast_data = applied_forecast.loc[applied_forecast['forecast [kWh]'] >= 0].resample('1D').sum()
        applied_forecast_consumption = applied_forecast.loc[applied_forecast['forecast [kWh]'] < 0].resample('1D').sum()

        applied_forecast_data.index = applied_forecast_data.index.date
        applied_forecast_consumption.index = applied_forecast_consumption.index.date

        applied_forecast_data = applied_forecast_data.T
        applied_forecast_consumption = applied_forecast_consumption.T
        applied_forecast_data.index.name = 'site'
        applied_forecast_data.index = [site]
        applied_forecast_consumption.index = [site]

        forecast_list.append(applied_forecast_data)
        consumption_forecast_list.append(applied_forecast_consumption)
        print('Ok!')
    
    forecast = pd.concat(forecast_list)
    consumption_forecast= pd.concat(consumption_forecast_list)

    forecast.to_excel(target_folder + f'applied_forecast_{target_year}_{target_month}.xlsx')
    consumption_forecast.to_excel(target_folder + f'applied_forecast_consumption_{target_year}_{target_month}.xlsx')

    # print(forecast)

RAW Forecast

In [ ]:
with engine.connect() as connection:

    forecast_list = list()
    consumption_forecast_list = list()

    for site in sites_data.keys():
        start = time.time()
        print('-'*50)
        print(site)
        site_data = dict()
        site_data['site'] = site
        site_data['site_id'], site_data['legal_entity'] = get_site_id(site, connection, 
                                                                      metadata.tables['sites'],
                                                                      include_legal_entity_id=True)
        mms=pd.DataFrame()
   
        first_date = dt.date(target_year, target_month, 1),
        last_date = dt.date(
            target_year, 
            target_month, 
            calendar.monthrange(target_year, target_month)[1]
        ) + dt.timedelta(days=7)

        applied_forecast = get_applied_forecast(
            site_data['site_id'], 
            first_date,
            last_date,
            connection=connection, 
            db_table=metadata.tables['forecasting_data'],
            forecast_type='forecast_applied_raw'
        )

        applied_forecast.index = applied_forecast.index.tz_localize(pytz.utc).tz_convert(pytz.timezone('europe/kiev'))
        # print(applied_forecast)
        applied_forecast_data = applied_forecast.loc[applied_forecast['forecast [kWh]'] >= 0].resample('1D').sum()
        applied_forecast_consumption = applied_forecast.loc[applied_forecast['forecast [kWh]'] < 0].resample('1D').sum()

        applied_forecast_data.index = applied_forecast_data.index.date
        applied_forecast_consumption.index = applied_forecast_consumption.index.date

        applied_forecast_data = applied_forecast_data.T
        applied_forecast_consumption = applied_forecast_consumption.T
        applied_forecast_data.index.name = 'site'
        applied_forecast_data.index = [site]
        applied_forecast_consumption.index = [site]

        forecast_list.append(applied_forecast_data)
        consumption_forecast_list.append(applied_forecast_consumption)
        print('Ok!')
    
    forecast = pd.concat(forecast_list)
    consumption_forecast= pd.concat(consumption_forecast_list)

    forecast.to_excel(target_folder + f'raw_forecast_{target_year}_{target_month}.xlsx')
    consumption_forecast.to_excel(target_folder + f'raw_forecast_consumption_{target_year}_{target_month}.xlsx')

    # print(forecast)